In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

In [2]:
# Carga de datos
df = pd.read_csv(r"../data/final/1_panel/final_dataset.csv")
df.insert(0, 'status_inf', df.pop('status_inf'))
df = df.drop(columns=['conglome', 'vivienda','hogar','codperso','periodo','ocupinf_t1','ocupinf_t2','periodo'])
df = df.dropna().reset_index(drop=True)
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24232 entries, 0 to 24231
Data columns (total 96 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   status_inf               24232 non-null  int64  
 1   dpto                     24232 non-null  int64  
 2   dominio                  24232 non-null  int64  
 3   tuvotrabajo              24232 non-null  int64  
 4   sector_trabajador        24232 non-null  int64  
 5   categoria_trabajador     24232 non-null  int64  
 6   trabajopara              24232 non-null  int64  
 7   registropersonajuridica  24232 non-null  float64
 8   cuentassunat             24232 non-null  float64
 9   pagosueldo               24232 non-null  int64  
 10  pagosalario              24232 non-null  int64  
 11  pagocomision             24232 non-null  int64  
 12  pagodestajo              24232 non-null  int64  
 13  pagosubvencion           24232 non-null  int64  
 14  pagohonorarios        

In [3]:
categoricas = ['sexo','estadocivil','tuvotrabajo','categoria_trabajador','sector_trabajador'
               ,'trabajopara','registropersonajuridica',
               'cuentassunat','tipocontrato','numpersonastrabajo','tuvootrotrabajo','normtrabaja',
               'disponiblehorastrabajar',
               'lenguamaterna','niveleduc','asiste_educ','usointernet','padece_enfer',
               'sintoma_malestar','enfermedad','recaida','accidente','notuvoenf',
               'puestosalud','centrosalud','centroopuesto','posta','hospital_minsa','hospital_seguro',
               'hospital_ffaa','consultorio','clinica','farmacia','domicilio','otro','no_busco',
               'no_dinero','lejos','demora_mucho','noconfia','nograve','remedio_caseros','no_seguro',
               'autoreceto','notiempo','maltrato','otro_motivo','essalud','seguroprivado',
               'eps','seguroffaa','sis','tipovivienda','materialparedes','materialpisos','materialtechos',
               'vivienda_status','viviendatitulo','agua_procedencia','conexionsshh','electricidad',
               'celular','internet','pobreza','dominio','dpto','ciiu_6c','area',
               'region','onp','afp',
               'pagosueldo','pagosalario','pagocomision','pagodestajo','pagosubvencion','pagohonorarios','pagoganancianegocio','pagoagropecuario','pagopropina','pagoespecie','pagootros','pagonorecibe']
# 'antepasadosconsidera','perteneceindig','registrosunat','deseaotrotrabajo','viviadistrito','agua_potable'
df_indep = df.drop(columns = ['status_inf'])
df_categoricas = df_indep[categoricas]


df_continuas = df_indep.drop(columns=categoricas)
print("Total de variables indep")
print(df_indep.shape[1])
print("La cantidad de variables indep categoricas:")
print(df_categoricas.shape[1])
print("La cantidad de variables indep continuas:")
print(df_continuas.shape[1])


Total de variables indep
95
La cantidad de variables indep categoricas:
83
La cantidad de variables indep continuas:
12


In [4]:
df_categoricas = df_categoricas.apply(lambda col: col.astype('int'))
print(df_categoricas.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24232 entries, 0 to 24231
Data columns (total 83 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   sexo                     24232 non-null  int64
 1   estadocivil              24232 non-null  int64
 2   tuvotrabajo              24232 non-null  int64
 3   categoria_trabajador     24232 non-null  int64
 4   sector_trabajador        24232 non-null  int64
 5   trabajopara              24232 non-null  int64
 6   registropersonajuridica  24232 non-null  int64
 7   cuentassunat             24232 non-null  int64
 8   tipocontrato             24232 non-null  int64
 9   numpersonastrabajo       24232 non-null  int64
 10  tuvootrotrabajo          24232 non-null  int64
 11  normtrabaja              24232 non-null  int64
 12  disponiblehorastrabajar  24232 non-null  int64
 13  lenguamaterna            24232 non-null  int64
 14  niveleduc                24232 non-null  int64
 15  as

In [5]:
df_continuas

,horas_normtrabaja,edad,habitaciones,habitacionesdormir,combustible,personas_ingresos,personas_hogar,ingtrabw,alquiler,tiempotrabajo,horastotales_sempasada,ratiodep
0,40,44,5.0,4.0,2.0,2,5,13577.0,400,9.583333,40,0.400000
1,45,26,5.0,4.0,6.0,4,5,18221.0,150,1.333333,45,0.800000
2,52,64,4.0,2.0,2.0,2,4,36028.0,500,29.666666,52,0.500000
3,54,72,10.0,1.0,2.0,1,2,26510.0,0,41.750000,48,0.500000
4,44,26,6.0,4.0,2.0,2,2,12113.0,100,0.500000,44,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
24227,60,61,6.0,5.0,2.0,4,7,24033.0,1000,17.000000,60,0.571429
24228,48,56,7.0,6.0,2.0,4,6,73477.0,1000,30.000000,48,0.666667
24229,42,29,7.0,6.0,2.0,4,6,42659.0,1000,2.000000,42,0.666667
24230,52,31,3.0,1.0,9.0,2,6,19657.0,18,0.250000,52,0.333333


In [6]:
percentil_99 = df_continuas['ingtrabw'].quantile(0.95)
df_continuas['ingtrabw'] = df_continuas['ingtrabw'].apply(lambda x: min(x, percentil_99))


In [ ]:

import numpy as np

# Reemplazar ceros por un valor pequeño (por ejemplo, 1e-10)
df_continuas['ingtrabw'] = df_continuas['ingtrabw'].replace(0, 1e-10)

# Ahora aplicar el logaritmo natural (base e)
df_continuas['ingtrabw'] = np.log(df_continuas['ingtrabw'])


In [33]:
# Crear dummies a partir de las variables categóricas
df_categoricas_encoded = pd.get_dummies(df_categoricas, columns=categoricas, drop_first=True, dummy_na=False)
print('La cantidad de variables indep categóricas después de haberse convertido en dummies es:')
print(df_categoricas_encoded.shape[1])

print("La cantidad de variables indep continuas:")
print(df_continuas.shape[1])

# El dataset final de variables independientes (categoricas convertidas a dummies + continuas)
df_indep_final = pd.concat([df_categoricas_encoded, df_continuas], axis=1)
print('La cantidad de variables indep totales (categoricas convertidas a dummies + continuas) es:')
print(df_indep_final.shape[1])

La cantidad de variables indep categóricas después de haberse convertido en dummies es:
229
La cantidad de variables indep continuas:
12
La cantidad de variables indep totales (categoricas convertidas a dummies + continuas) es:
241


In [34]:
df_continuas

,horas_normtrabaja,edad,habitaciones,habitacionesdormir,combustible,personas_ingresos,personas_hogar,ingtrabw,alquiler,tiempotrabajo,horastotales_sempasada,ratiodep
0,40,44,5.0,4.0,2.0,2,5,9.516132,400,9.583333,40,0.400000
1,45,26,5.0,4.0,6.0,4,5,9.810330,150,1.333333,45,0.800000
2,52,64,4.0,2.0,2.0,2,4,10.492052,500,29.666666,52,0.500000
3,54,72,10.0,1.0,2.0,1,2,10.185277,0,41.750000,48,0.500000
4,44,26,6.0,4.0,2.0,2,2,9.402035,100,0.500000,44,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
24227,60,61,6.0,5.0,2.0,4,7,10.087183,1000,17.000000,60,0.571429
24228,48,56,7.0,6.0,2.0,4,6,11.191391,1000,30.000000,48,0.666667
24229,42,29,7.0,6.0,2.0,4,6,10.660994,1000,2.000000,42,0.666667
24230,52,31,3.0,1.0,9.0,2,6,9.886189,18,0.250000,52,0.333333


In [35]:
# Filtrar variables con un gran número de faltantes
null_percent = df_indep_final.isnull().mean()*100
cols_null = null_percent[null_percent > 80 ].index
cols_validas = null_percent[null_percent <= 80 ].index
print(cols_null)
# df_null = df[cols_validas].copy()
# df_null.shape[1]

Index([], dtype='object')


In [36]:
# Para eliminar las variables con muy poca varianza (definido por el umbral)
# Asegurar que todas las columnas dummy sean numéricas
df_indep_final = df_indep_final.apply(pd.to_numeric)

varianzas = df_indep_final.var()
umbral = 0.01
vars_seleccionadas = varianzas[varianzas > umbral].index
vars_no_seleccionadas = varianzas[varianzas <= umbral].index

print("Las variables con varianza menor al umbral son:")
print(df_indep_final[vars_no_seleccionadas].nunique())
## print(df_indep_final[vars_seleccionadas].nunique())

df_indep_final=df_indep_final[vars_seleccionadas]

print("El nuevo número de variables del dataset de variables independientes es:")
print(df_indep_final.shape[1])

Las variables con varianza menor al umbral son:
categoria_trabajador_6    2
trabajopara_3             2
tipocontrato_3            2
tipocontrato_4            2
tipocontrato_5            2
lenguamaterna_3           2
lenguamaterna_5           2
lenguamaterna_6           2
lenguamaterna_7           2
lenguamaterna_8           2
centrosalud_1             2
centroopuesto_1           2
hospital_ffaa_1           2
domicilio_1               2
otro_1                    2
no_dinero_1               2
lejos_1                   2
no_seguro_1               2
maltrato_1                2
tipovivienda_5            2
tipovivienda_6            2
tipovivienda_7            2
materialparedes_2         2
materialparedes_6         2
materialparedes_8         2
materialpisos_7           2
materialtechos_6          2
materialtechos_7          2
materialtechos_8          2
vivienda_status_5         2
vivienda_status_7         2
agua_procedencia_3        2
electricidad_3            2
pagocomision_1            2


In [37]:
import numpy as np

# Calcular la matriz de correlación
corr_matrix = df_indep_final.corr().abs()

# Seleccionar la parte superior de la matriz de correlación (sin la diagonal)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Identificar las columnas a eliminar por correlación alta
to_drop = []
correlations_to_show = []

for column in upper.columns:
    high_corr = upper[column] > 0.50  # Umbral de 0.70
    if any(high_corr):
        to_drop.append(column)
        correlations_to_show.append((column, upper[column][high_corr]))

# Mostrar las correlaciones altas y las variables a eliminar
print("Correlaciones altas que exceden el umbral de 0.70:")
for column, corr_values in correlations_to_show:
    print(f"\nColumna: {column}")
    print(corr_values)

print("\nVariables a eliminar por alta correlación:")
print(to_drop)

# Eliminar las variables con alta correlación
df_indep_final = df_indep_final.drop(columns=to_drop)

# Mostrar el nuevo número de variables después de la eliminación
print("\nNuevo número de variables del dataset de variables independientes después de eliminar las que tienen una alta correlación:")
print(df_indep_final.shape[1])

Correlaciones altas que exceden el umbral de 0.70:

Columna: categoria_trabajador_4
categoria_trabajador_3    0.568657
Name: categoria_trabajador_4, dtype: float64

Columna: sector_trabajador_9
categoria_trabajador_4    0.603384
Name: sector_trabajador_9, dtype: float64

Columna: trabajopara_6
trabajopara_2    0.5421
Name: trabajopara_6, dtype: float64

Columna: trabajopara_99
categoria_trabajador_2    0.769422
categoria_trabajador_3    0.621682
Name: trabajopara_99, dtype: float64

Columna: registropersonajuridica_2
categoria_trabajador_2    0.716144
categoria_trabajador_3    0.521312
trabajopara_99            0.809704
Name: registropersonajuridica_2, dtype: float64

Columna: registropersonajuridica_99
trabajopara_2    0.923276
trabajopara_6    0.587148
Name: registropersonajuridica_99, dtype: float64

Columna: cuentassunat_2
categoria_trabajador_2       0.677869
trabajopara_99               0.710342
registropersonajuridica_2    0.742730
Name: cuentassunat_2, dtype: float64

Columna: 

In [38]:
df_model_xgb_rf = pd.concat([df[["status_inf"]], df_indep_final], axis=1)

df_model_xgb_rf.to_csv(r'../data/final/1_panel/3_modelling/df_model_post_tratamiento_estadistico.csv', index=False)


In [39]:
df_model_xgb_rf

,status_inf,sexo_2,estadocivil_2,estadocivil_3,estadocivil_4,estadocivil_5,estadocivil_6,tuvotrabajo_2,categoria_trabajador_2,categoria_trabajador_3,...,onp_1,pagoespecie_1,horas_normtrabaja,edad,habitaciones,combustible,personas_ingresos,ingtrabw,alquiler,ratiodep
0,0,True,True,False,False,False,False,False,False,True,...,False,True,40,44,5.0,2.0,2,9.516132,400,0.400000
1,0,False,False,False,False,False,True,False,False,True,...,False,False,45,26,5.0,6.0,4,9.810330,150,0.800000
2,0,False,True,False,False,False,False,False,True,False,...,False,False,52,64,4.0,2.0,2,10.492052,500,0.500000
3,0,False,True,False,False,False,False,False,False,False,...,False,False,54,72,10.0,2.0,1,10.185277,0,0.500000
4,0,False,False,False,False,False,True,False,False,True,...,False,False,44,26,6.0,2.0,2,9.402035,100,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24227,1,False,True,False,False,False,False,False,True,False,...,False,False,60,61,6.0,2.0,4,10.087183,1000,0.571429
24228,0,False,True,False,False,False,False,False,False,False,...,False,True,48,56,7.0,2.0,4,11.191391,1000,0.666667
24229,0,False,False,False,False,False,True,False,False,True,...,False,False,42,29,7.0,2.0,4,10.660994,1000,0.666667
24230,0,False,True,False,False,False,False,False,False,False,...,False,True,52,31,3.0,9.0,2,9.886189,18,0.333333
